This notebook uses the Dept. Ag, Water and the Environment's data on Species of National Environmental Significance
See https://environment.gov.au/science/erin/databases-maps/snes
and http://www.environment.gov.au/fed/catalog/search/resource/details.page?uuid=%7B337B05B6-254E-47AD-A701-C55D9A0435EA%7D

Specifically a subset of the "critically endangered" mammals listed in the dataset - Leadbetter Possum

# Install requisite software libraries

In [1]:
!pip install --quiet git+https://github.com/GeoscienceAustralia/AusPIX_DGGS.git

In [2]:
!pip install geojson pygeoj

Load the "critically_endangered_mammals_leadbetter.geojson" data file

In [3]:
import geojson
fname = "critically_endangered_mammals_leadbetter.geojson"
geojson_data = None
with open(fname) as json_file:
    geojson_data = geojson.load(json_file)
    

Show the locations of critically endangered leadbetter possums on a map

In [4]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -37.8
y_coord = 145.89

map1 = ipy.Map(center=[x_coord, y_coord], zoom=8)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'red', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.5})
map1.add_layer(geo_json1)


In [5]:
## Critically endangered mammals: leadbetter Possum
map1

Map(center=[-37.8, 145.89], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [6]:
from pandas import json_normalize
features = geojson_data['features']
json_normalize(features)

,type,geometry.type,geometry.coordinates,properties.OBJECTID,properties.Shape_Leng,properties.Shape_Area,properties.listed_id,properties.map_id,properties.sci_name,properties.comm_name,...,properties.tax_king,properties.other_ids,properties.cell_size,properties.regions,properties.attributio,properties.sprat_prof,properties.style.color,properties.style.opacity,properties.style.weight,properties.style.fillOpacity
0,Feature,MultiPolygon,"[[[[145.51, -37.82], [145.51, -37.83], [145.52...",173,8.04,0.1937,273,273,Gymnobelideus leadbeateri,Leadbeater's Possum,...,Animalia,None,0.01,VIC,Species 1km Grids August 2019 Canberra: Depart...,http://www.environment.gov.au/cgi-bin/sprat/pu...,red,0.8,1.9,0.5
1,Feature,MultiPolygon,"[[[[145.55, -37.89], [145.55, -37.92], [145.52...",174,13.68,0.2404,273,273,Gymnobelideus leadbeateri,Leadbeater's Possum,...,Animalia,None,0.01,VIC,Species 1km Grids August 2019 Canberra: Depart...,http://www.environment.gov.au/cgi-bin/sprat/pu...,red,0.8,1.9,0.5


In [7]:
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly
from auspixdggs.callablemodules.dggs_in_line import line_to_DGGS, densify_my_line
import geojson
from geojson.utils import coords
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon

def get_geojson_by_file(fname):
    data = None
    with open(fname) as json_file:
        data = geojson.load(json_file)
    return data

def geojson_to_shape(g):
    return shape(g)
    
def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     #ret = f"({box[0][0]} {box[1][0]}, {box[0][1]} {box[1][1]})"
     ret = [ box[0][0], box[1][0], box[0][1], box[1][1] ]
     #print("BBOX {}".format(ret))
     return ret

def get_cells_in_geojson(geojson, resolution, cell_obj=True):
    list_cells = []
    #print("Type geojson: {}".format(type(geojson)))
    dict_cells = {}
    for fea in geojson['features']:  # for feature in attribute table
        #print(fea)
        res_cells = get_cells_in_feature(fea, resolution, cell_obj=cell_obj)
        print(res_cells)
        for item in res_cells:
            print("'{}'".format(str(item)))
        #cell_list = [item[0] for item in res_cells]
        cell_list = res_cells
        #cell_id_list = res_cells
        for cell in cell_list:
            cell_id = str(cell)
            if cell_id not in dict_cells:
                dict_cells[cell_id] = cell
    list_cells = list(dict_cells.values())
    return list_cells 

def get_cells_in_feature(fea, resolution, cell_obj=True):
    geom = geojson_to_shape(fea['geometry'])
    curr_coords = list(coords(fea))
    thisbbox = bbox(curr_coords)
    cells = []
    if isinstance(geom, LineString) or isinstance(geom, MultiLineString): 
        fea['geometry']['coordinates'] = densify_my_line(fea['geometry']['coordinates'], resolution)
        curr_coords = list(coords(fea))
        print(curr_coords)        
        res_cells = line_to_DGGS(curr_coords, resolution)  # start at DGGS level 10   
        cells = [str(item) for item in res_cells]
    elif isinstance(geom, Polygon) or  isinstance(geom, MultiPolygon):
        res_cells = cells_in_poly(thisbbox, fea['geometry']['coordinates'], resolution, return_cell_obj=cell_obj)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    else: #try something anyway
        res_cells = cells_in_poly(thisbbox, fea['geometry']['coordinates'], resolution, return_cell_obj=cell_obj)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    return cells   

# Use the Geoscience Australia AusPIX DGGS Engine to DGGS-enable the data

Use the DGGS Engine to find cells that match the features. Also specify the DGGS resolution needed.

In [8]:
DGGS_RESOLUTION=7

In [9]:
list_cells = get_cells_in_geojson(geojson_data, DGGS_RESOLUTION, True)  

auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix

num cells in bb =  384
total vertex in this poly 8
new poly [[145.51, -37.82], [145.51, -37.83], [145.52, -37.83], [145.52, -37.85], [145.54, -37.85], [145.54, -37.86], [145.55, -37.86], [145.55, -37.89], [145.54, -37.89], [145.51, -37.89], [145.51, -37.87], [145.5, -37.87], [145.49, -37.87], [145.49, -37.86], [145.48, -37.86], [145.48, -37.83], [145.48, -37.81], [145.49, -37.81], [145.49, -37.8], [145.51, -37.8], [145.51, -37.82]]
[145.51, -37.82]
[145.51, -37.83]
[145.52, -37.83]
[145.52, -37.85]
[145.54, -37.85]
[145.54, -37.86]
[145.55, -37.86]
[145.55, -37.89]
[1

'R7848086'
'R7848087'
'R7848088'
'R7848166'
'R7848321'
'R7848322'
'R7848400'
'R7847525'
'R7848403'
'R7847526'
'R7847527'
'R7847528'
'R7848306'
'R7848328'
'R7847551'
'R7847552'
'R7848330'
'R7848331'
'R7848430'
'R7848450'
'R7848451'
'R7847555'
'R7848333'
'R7848334'
'R7848433'
'R7848434'
'R7848435'
'R7848336'
'R7848346'
'R7848437'
'R7848362'
'R7848370'
'R7848371'
'R7848372'
'R7848470'
'R7848471'
'R7848472'
'R7848480'
'R7848481'
'R7848364'
'R7848365'
'R7848373'
'R7848374'
'R7848375'
'R7848474'
'R7848475'
'R7848484'
'R7848367'
'R7848368'
'R7848376'
'R7848387'
'R7848388'
'R7848466'
'R7848468'
'R7848476'
'R7848477'
'R7848478'
'R7848486'
'R7848487'
'R7848566'
'R7847821'
'R7848610'
'R7848611'
'R7848612'
'R7848620'
'R7848621'
'R7848622'
'R7848700'
'R7848701'
'R7848702'
'R7848710'
'R7848711'
'R7848712'
'R7848720'
'R7848721'
'R7848800'
'R7848801'
'R7847824'
'R7848614'
'R7848615'
'R7848623'
'R7848624'
'R7848625'
'R7848703'
'R7848704'
'R7848713'
'R7848714'
'R7848723'
'R7848724'
'R7848725'
'R7848803'

[[<auspixdggs.auspixengine.dggs.Cell object at 0x7ff9154ec390>, 145.8230452674897, -37.31893851494384], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff9154ec290>, 145.8641975308642, -37.31893851494384], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff9154ec4d0>, 145.94650205761317, -37.31893851494384], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff9154f7390>, 145.78189300411523, -37.36286475857879], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff915500cd0>, 145.98765432098764, -37.36286475857879], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff9154aa890>, 145.61728395061726, -37.406816512623934], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff915504c50>, 145.74074074074073, -37.406816512623934], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff9154aa450>, 145.78189300411523, -37.406816512623934], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff9155043d0>, 145.8230452674897, -37.406816512623934], [<auspixdggs.auspixengine.dggs.Cell object at 0x7ff914de7990>,

In [10]:
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from geojson import Feature, FeatureCollection

list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon(bbox_coords)
    feat = Feature(geometry=geom_obj, properties={"dggs_cell_id": str(cell)}) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)

Show the matching DGGS Cells on a map

In [11]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=9)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=feature_collection, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'blue', 
                          'fillOpacity': 0.2})
map2.add_layer(geo_json1)
map2.add_layer(geo_json2)



In [12]:
# DGGS Enabled version - Critically endangered mammals: leadbetter Possum

map2


Map(center=[-37.8, 145.89], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

# Query the Loc-I Integration API for matching LGAs

In [13]:
import requests
import json
#iterate over features in feature collection and query Loc-I API for features
def find_features_at_dggs_cell(dggs_cell_id):
    payload = {
        'dggs_cell' : dggs_cell_id
    }
    r = requests.get('https://api.loci.cat/api/v1/location/find-at-DGGS-cell', params=payload)
    json_res = r.json()
    return json_res


In [14]:
target_feature_type = 'asgs16_lga'

set_of_lga_features = set([])
#print(feature_collection)
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    #pad it out
    cell_id = cell_id + "000"
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_lga_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))

    


Processing R7848085000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7848086000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7848087000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7848088000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7848166000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7848321000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7848322000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7848400000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7847525000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing R7848403000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/25620
Processing

Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848704000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848713000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848714000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848723000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848724000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848725000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848803000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848804000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848805000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848616000
Adding htt

Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/27450
Processing R7848446000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/27450
Processing R7848447000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/27450
Processing R7848448000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/27450
Processing R7848456000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848457000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20830
Processing R7848458000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/24250
Processing R7848360000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/27450
Processing R7848361000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/27450
Processing R7848380000
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/27450
Processing R7848381000
Adding htt

In [15]:
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
    return geojson_data



Show the matching LGAs on a map

In [16]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=7)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


In [17]:
for feature_uri in list(set_of_lga_features):
    geojson_data = get_geom(feature_uri)
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'yellow', 
                          'fillOpacity': 0.3})
    map3.add_layer(curr_geojson_layer)
map3.add_layer(geo_json1)
map3.add_layer(geo_json2)


In [18]:
map3

Map(center=[-37.8, 145.89], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

# Try a different Loc-I Feature - Query the Loc-I Integration API for matching ASGS Statistical Area Level 2 locations

In [19]:
target_feature_type = 'asgs16_sa2'

set_of_features = set([])
#print(feature_collection)
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    #pad it out
    cell_id = cell_id + "000"
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))


Processing R7848085000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011054
Processing R7848086000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011054
Processing R7848087000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011054
Processing R7848088000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011054
Processing R7848166000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011054
Processing R7848321000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011054
Processing R7848322000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011054
Processing R7848400000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011054
Processing R7847525000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011062
Processing R7848403000
Adding http://linked.data.gov.au

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848703000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848704000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848713000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848714000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848723000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848724000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848725000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848803000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848804000
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011061
Processing R7848438000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011061
Processing R7848446000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011061
Processing R7848447000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011061
Processing R7848448000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011061
Processing R7848456000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848457000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/205011077
Processing R7848458000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/204011057
Processing R7848360000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/211051276
Processing R7848361000
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Show the matching ASGS SA2 on a map

In [20]:

map4 = ipy.Map(center=[x_coord, y_coord], zoom=8)


for feature_uri in list(set_of_features):
    geojson_data = get_geom(feature_uri)
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'green', 
                          'fillOpacity': 0.3})
    map4.add_layer(curr_geojson_layer)

    
map4.add_layer(geo_json1)
map4.add_layer(geo_json2)

In [21]:
map4

Map(center=[-37.8, 145.89], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

# Try a different Loc-I Feature-type again - Query the Loc-I Integration API for matching ASGS State Suburbs

In [22]:
target_feature_type = 'asgs16_ssc'

set_of_features = set([])
#print(feature_collection)
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    #pad it out
    cell_id = cell_id + "000"
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))


Processing R7848085000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22198
Processing R7848086000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22198
Processing R7848087000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22198
Processing R7848088000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22198
Processing R7848166000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20844
Processing R7848321000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22198
Processing R7848322000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22198
Processing R7848400000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20844
Processing R7847525000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/21013
Processing R7848403000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20844
Processing R7847526000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22523

Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20166
Processing R7848804000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20166
Processing R7848805000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22493
Processing R7848616000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22501
Processing R7848617000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20988
Processing R7848618000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22103
Processing R7848626000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22103
Processing R7848627000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20006
Processing R7848628000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20006
Processing R7848706000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/21942
Processing R7848727000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22430
Processing R7848728000

Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/21257
Processing R7848363000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/21802
Processing R7848383000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20827
Processing R7848384000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/21620
Processing R7848385000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22147
Processing R7848463000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22147
Processing R7848464000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/22147
Processing R7848465000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20450
Processing R7848473000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/20450
Processing R7848483000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/21257
Processing R7848485000
Adding http://linked.data.gov.au/dataset/asgs2016/statesuburb/21257
Processing R7848377000

Show the matching State Suburbs on a map

In [23]:

map5 = ipy.Map(center=[x_coord, y_coord], zoom=8)


for feature_uri in list(set_of_features):
    geojson_data = get_geom(feature_uri)
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'purple', 
                          'fillOpacity': 0.3})
    map5.add_layer(curr_geojson_layer)

    
map5.add_layer(geo_json1)
map5.add_layer(geo_json2)

In [24]:
map5

Map(center=[-37.8, 145.89], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…